<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Gathering Intelligence with Generative AI   

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.30.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.30.0-alpha"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info using o1 series of models
var azureOpenAIEndpoint = config["AzureOpenAI:o1Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:o1APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:o1ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.30.0 Microsoft.SemanticKernel.Plugins.Core, 1.30.0-alpha

In [ ]:
using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Using Azure OpenAI Service


### Step 2 - Understanding Built-In Decision Intelligence with Generative AI

Generative AI models are trained on very large amounts of data. This data includes decision-making techniques, decision frameworks, reasoning techniques, decision best practices and other adjacent decision data that can help make decisions. Therefore, an obvious step in gathering Decision Intelligence that built-into the GenAI model is important to understand.  

Certain models (typically those that have more parameters) have more "intelligence built-in". Furthermore, models that specialize in certain tasks or a fine-tuned for a specific domain can also have more intelligence built-in. A great example of model specialization is the OpenAI o1 series of models. The o1 series of models are advanced reasoning models that are a great fit for decision-making. Therefore, it is a good hypotheses to test if these o1 models have additional Decision Intelligence data built-in.  

Execute the cell below to view the prompt output asking for a "complete" list of Decision Frameworks built-into the o1 series of models.

In [11]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPrompt = """
Identify and a very complete list of decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

Create a column for the Number (in ascending order), framework name and decsription.
Surface the gathered frameworks in a Markdown table format. Do not surround the markdown with triple backticks.
Only return the Markdown table no other text.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {decisionAnalysisPrompt}
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

| Number | Framework Name                  | Description                                                                                                                                                                                                                             |
|--------|---------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1      | SWOT Analysis                   | Evaluates Strengths, Weaknesses, Opportunities, and Threats to understand internal and external factors affecting a decision.                                                                                                            |
| 2      | Decision Tree Analysis          | Uses a tree-like model to map out options and their possible outcomes, helping to choose the most beneficial path.   

| Number | Framework Name            | Description                                                                                                                                                                  |
|--------|---------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1      | SWOT Analysis             | Identifies Strengths, Weaknesses, Opportunities, and Threats to assess internal and external factors, enhancing strategic planning and decision-making.                       |
| 2      | Decision Matrix           | Evaluates and prioritizes options by scoring them based on set criteria, bringing objectivity to decisions involving multiple factors.                                        |
| 3      | Cost-Benefit Analysis     | Compares expected costs and benefits of actions to determine financial feasibility and profitability, aiding in economic decision-making.                                    |
| 4      | Pareto Analysis           | Applies the 80/20 rule to identify the most significant factors contributing to an outcome, helping prioritize issues with the greatest impact when addressed.               |
| 5      | Six Thinking Hats         | Assigns different perspectives (hats) to approach decisions from various angles, encouraging comprehensive analysis and creative thinking in group discussions.              |
| 6      | OODA Loop                 | Stands for Observe, Orient, Decide, Act; a cyclical process emphasizing agility and situational awareness, useful in fast-paced and competitive scenarios.                   |
| 7      | PDCA Cycle                | Plan-Do-Check-Act; a continuous improvement framework that implements changes systematically, ensuring decisions are tested and refined through iterations.                   |
| 8      | Root Cause Analysis       | Identifies the fundamental causes of problems rather than just symptoms, leading to more effective and lasting solutions.                                                     |
| 9      | Fishbone Diagram          | Visually maps out potential causes of a problem, facilitating thorough analysis and problem-solving by categorizing contributing factors.                                     |
| 10     | PEST Analysis             | Assesses Political, Economic, Social, and Technological external factors impacting decisions or organizations, aiding in strategic planning and environmental scanning.        |
| 11     | Kepner-Tregoe Method      | Provides a structured approach for gathering information, prioritizing factors, and systematically choosing the best option in complex decisions.                             |
| 12     | Force Field Analysis      | Identifies driving and restraining forces affecting a decision, helping visualize factors that can impact success and guide strategies to strengthen or weaken these forces.  |
| 13     | Risk Analysis             | Involves identifying and assessing potential risks and uncertainties in decisions, allowing for better preparation and mitigation strategies.                                 |
| 14     | Decision Trees            | Uses a graphical representation of choices and their possible consequences, including chance event outcomes, resource costs, and utility, to aid in selecting the most beneficial option. |
| 15     | Analytic Hierarchy Process (AHP) | Breaks down complex decisions into a hierarchy of more manageable parts, using pairwise comparisons and priority scaling to determine the best option based on multiple criteria. |
| 16     | Scenario Analysis         | Explores and analyzes possible future events by considering alternative plausible scenarios, assisting in strategic planning and risk management under uncertainty.          |
| 17     | Delphi Technique          | Utilizes rounds of questionnaires sent to experts to gather opinions and reach a consensus on complex issues, enhancing decision-making with expert insights.                  |
| 18     | SMART Goals               | Ensures goals are Specific, Measurable, Achievable, Relevant, and Time-bound, facilitating clearer objectives and effective planning.                                         |
| 19     | Monte Carlo Simulation    | Uses random sampling and statistical modeling to estimate mathematical functions and mimic the operations of complex systems, assessing risk and uncertainty in decision-making. |
| 20     | Bayes' Theorem            | Applies probabilistic reasoning to update the probability for a hypothesis as more evidence or information becomes available, enhancing decisions under uncertainty.           |
| 21     | Game Theory               | Analyzes strategic interactions where the outcome for each participant depends on the actions of others, aiding in competitive and cooperative decision-making scenarios.      |
| 22     | Value Chain Analysis      | Examines an organization's activities to understand how they contribute to value creation, helping identify areas for improvement and competitive advantage.                   |
| 23     | Balanced Scorecard        | Integrates financial and non-financial performance measures to provide a more comprehensive view of organizational performance against strategic goals.                       |
| 24     | The Ladder of Inference   | Describes the mental process of moving from data to decisions, highlighting the importance of testing assumptions and reasoning to avoid flawed conclusions.                  |
| 25     | Pros-Cons Analysis        | Lists the advantages and disadvantages of each option, facilitating straightforward comparison and aiding in rational decision-making.                                        |
| 26     | Effective Altruism Framework | Utilizes evidence and reasoning to determine the most effective ways to benefit others, guiding philanthropic and ethical decisions toward the greatest impact.            |
| 27     | Eisenhower Matrix         | Prioritizes tasks based on urgency and importance, helping focus on what matters most and effectively manage time by categorizing tasks into four quadrants.                   |
| 28     | The Cynefin Framework     | Helps understand the context of a situation (simple, complicated, complex, chaotic) to make appropriate decisions and apply suitable management approaches.                    |
| 29     | Systems Thinking          | Views problems as part of an overall system, focusing on the interactions and interdependencies rather than isolated components, leading to more comprehensive solutions.       |
| 30     | Cognitive Bias Awareness  | Recognizes and mitigates cognitive biases that can impair judgment, leading to more rational, objective decisions by acknowledging personal and systemic biases.               |
| 31     | RACI Matrix               | Clarifies roles and responsibilities by defining who is Responsible, Accountable, Consulted, and Informed in decision-making processes, enhancing communication and accountability. |
| 32     | Causal Loop Diagrams      | Uses visual representations of systems to show feedback loops and interactions, aiding in understanding complex systems and predicting behavior over time.                     |
| 33     | TRIZ (Theory of Inventive Problem Solving) | Provides a systematic approach for solving complex problems by analyzing patterns in innovative solutions across industries.                             |
| 34     | MOSCOW Method             | Prioritizes requirements or tasks by categorizing them as Must-have, Should-have, Could-have, and Won't-have, assisting in resource allocation and scope management.           |
| 35     | SCQA Framework            | Stands for Situation, Complication, Question, and Answer; helps structure communication and decision narratives effectively, ensuring clarity and logical flow.                |
| 36     | Fail-Safing (Poka-Yoke)   | Designs processes to prevent errors or mitigate their effects, enhancing decision robustness and operational reliability by anticipating potential failures.                   |
| 37     | Benchmarking              | Compares practices and performance metrics with industry leaders or competitors to identify areas for improvement and set performance goals.                                   |
| 38     | Stakeholder Analysis      | Identifies stakeholders' interests, influence, and impact on decisions, aiding in effective communication, conflict resolution, and ensuring decisions meet key stakeholders' needs. |
| 39     | Critical Path Method (CPM)| Determines the sequence of essential tasks that determine the minimum project duration, optimizing scheduling, resource allocation, and timely decision-making in projects.     |
| 40     | Ethical Decision-Making Models | Incorporate ethical principles and values into the decision process, ensuring choices are aligned with moral and societal standards, and considering the broader impact.       |
| 41     | Nudge Theory              | Applies behavioral economics insights to subtly influence choices and behaviors, improving decision outcomes without restricting options by altering the decision environment.    |
| 42     | Red Teaming               | Involves adopting an adversarial approach to challenge strategies and decisions, exposing vulnerabilities, and enhancing robustness by anticipating opposition or failure modes. |
| 43     | Design Thinking           | Employs a human-centered approach to innovation, integrating the needs of people, possibilities of technology, and requirements for business success through iterative prototyping and testing. |
| 44     | Empathy Mapping           | Uses visual tools to understand users' experiences and perspectives, promoting decisions that are aligned with stakeholders' needs and enhancing user-centered design.            |
| 45     | McKinsey 7S Framework     | Analyzes seven internal organizational elements (Strategy, Structure, Systems, Shared Values, Skills, Style, Staff) to ensure alignment and effectiveness in decision-making and change management. |
| 46     | Porter's Five Forces      | Examines competitive forces within an industry (Competitive Rivalry, Supplier Power, Buyer Power, Threat of Substitution, Threat of New Entry) to inform strategic decisions and market positioning. |
| 47     | Blue Ocean Strategy       | Encourages creating new market spaces (blue oceans) rather than competing in existing ones (red oceans), promoting differentiation and uncontested market creation.              |
| 48     | Appreciative Inquiry      | Focuses on identifying and leveraging an organization's strengths and successes to drive change and innovation, enhancing positive decision-making and organizational development. |
| 49     | Mind Mapping              | Utilizes visual diagrams to organize thoughts, ideas, and information, facilitating brainstorming, creativity, and problem-solving by illustrating relationships and hierarchies.   |
| 50     | Lean Six Sigma            | Combines Lean methodologies and Six Sigma principles to improve efficiency, reduce waste, and enhance quality in processes and decisions through data-driven approaches and continuous improvement. |


The prompt paired with the o1 model above likely returned a list of 30-50 (depending on randomness) Decision Frameworks it is aware of. Presumably, because it described these decision frameworks, it should be able to operate on them to make decisions. This highlights just some of decision-making capability of the o1 series of model. What if there are more decision frameworks that are available? This is potentially just a list of the top frameworks (and certainly not complete) of frameworks the model has identified. There could be a great deal more!

The code below will iterate several times (5x) and continue to prompt the GenAI model with the same instruction until a more comprehensive list is built. Notice the output below can have well over 100 decision-frameworks that the o1 series model is aware of.

In [12]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var markDownTaleOfDecisionFrameworks = string.Empty;

// This can quite a long time. View the output results below to see the final markdown table
// Note: This is done for illustrative purposes. 
// Ideally, the markdown formatting and combining of tables should be done in seperate steps for optimal results.
for (int i = 0; i != 5; i++)
{
    var systemPromptForDecisions = $"""
    You are a Decision Intelligence Assistant. 
    Assist the user in exploring options, reasoning through decisions, problem-solving.
    Apply systems thinking to various scenarios. 
    Provide structured, logical, and comprehensive advice.
    """;
    var decisionAnalysisPrompt = $"""
    Identify and a very comprehensive list of decision-making frameworks that can enhance the quality of decisions. 
    Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

    Surface the gathered frameworks in a Markdown table format. Do not surround the markdown with triple backticks.
    The FOUR table columns should be: 
    Number (in ascending order), Decision-Framework Name, 
    Value (a number between 1 to 10, how valueable the framework is) and Description.
    Only return the Markdown table no other text.

    This prompt is going to be called several times to gather a full list of decision-making frameworks. 
    The previous Markdown table is located in the area surrounded by <DecisionMakingFrameworks> tags.
    Make sure to copy the table found in the previous responses and append new unique frameworks to the table. 
    DO NOT discard the previous Markdown table found in the <DecisionMakingFrameworks> tags.
    Add new unique frameworks by appending to the previous Markdown table. Do not recreate the table header.
    The final Markdown table output should include both:
    The previous markdown table (found in the <DecisionMakingFrameworks> Tags) combined with new unique frameworks output in a single table.

    <DecisionMakingFrameworks>
    {markDownTaleOfDecisionFrameworks}
    </DecisionMakingFrameworks>.
    """;

    // How the prompt looks like to the LLM
    var simpleDecisionPromptTemplate = $"""
    System Prompt: {systemPromptForDecisions}

    Request from the user: {decisionAnalysisPrompt}
    """;

    // Uncomment this line to view the debugging 
    // Console.WriteLine(decisionAnalysisPrompt);

    var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
    // This will append the new markdown table to the previous markdown tables
    markDownTaleOfDecisionFrameworks = simplePromptResponse.GetValue<string>();
    Console.WriteLine($"Finished Gethering Intelligence Iteration: {i + 1}");
}

// Write the final markdown table to the console
Console.WriteLine(markDownTaleOfDecisionFrameworks);

Finished Gethering Intelligence Iteration: 1
Finished Gethering Intelligence Iteration: 2
Finished Gethering Intelligence Iteration: 3
Finished Gethering Intelligence Iteration: 4
Finished Gethering Intelligence Iteration: 5
| Number | Decision-Framework Name | Value | Description |
|--------|-------------------------|-------|-------------|
| 1 | SWOT Analysis | 8 | Identifies Strengths, Weaknesses, Opportunities, and Threats to assess internal and external factors affecting a decision. |
| 2 | Decision Matrix Analysis | 9 | Evaluates and prioritizes a list of options by scoring them against different criteria. |
| 3 | Cost-Benefit Analysis | 9 | Compares the costs and benefits of options to determine their overall value or feasibility. |
| 4 | Pareto Analysis | 7 | Uses the 80/20 rule to prioritize decisions that will have the greatest impact. |
| 5 | Risk Analysis | 8 | Identifies potential risks and evaluates their impact to mitigate negative outcomes. |
| 6 | Six Thinking Hats | 8 

| Number | Decision-Framework Name | Value | Description |
|--------|-------------------------|-------|-------------|
| 1 | SWOT Analysis | 8 | Identifies Strengths, Weaknesses, Opportunities, and Threats to assess internal and external factors affecting a decision. |
| 2 | Decision Matrix Analysis | 9 | Evaluates and prioritizes a list of options by scoring them against different criteria. |
| 3 | Cost-Benefit Analysis | 9 | Compares the costs and benefits of options to determine their overall value or feasibility. |
| 4 | Pareto Analysis | 7 | Uses the 80/20 rule to prioritize decisions that will have the greatest impact. |
| 5 | Risk Analysis | 8 | Identifies potential risks and evaluates their impact to mitigate negative outcomes. |
| 6 | Six Thinking Hats | 8 | Encourages looking at a problem from multiple perspectives to enhance creativity and decision quality. |
| 7 | OODA Loop | 8 | A cyclic process of Observe, Orient, Decide, and Act for rapid decision-making. |
| 8 | PDCA Cycle | 7 | Plan, Do, Check, Act; a continuous loop for process improvement and effective decision-making. |
| 9 | Eisenhower Matrix | 8 | Prioritizes tasks based on urgency and importance to improve time management and decision focus. |
| 10 | Vroom-Yetton Decision Model | 7 | Guides leaders to the optimal decision-making approach based on the situation and team dynamics. |
| 11 | Root Cause Analysis | 8 | Identifies the underlying causes of problems to address them effectively. |
| 12 | Delphi Method | 6 | Gathers expert opinions through rounds of questionnaires to reach a consensus decision. |
| 13 | PMI (Plus, Minus, Interesting) | 6 | Evaluates the positives, negatives, and interesting points of an option to make balanced decisions. |
| 14 | Monte Carlo Simulation | 7 | Uses statistical modeling to predict the probability of different outcomes in uncertain situations. |
| 15 | Rational Decision-Making Model | 8 | Follows a structured and logical process to make optimal decisions based on data and analysis. |
| 16 | Recognition-Primed Decision Model | 7 | Combines intuition and analysis for quick decision-making in complex situations based on experience. |
| 17 | De Bono's Lateral Thinking | 8 | Promotes creative problem-solving by thinking outside traditional patterns. |
| 18 | Game Theory | 8 | Analyzes competitive situations where the outcomes depend on the actions of all participants. |
| 19 | Incrementalism | 6 | Advocates for making decisions through small, manageable steps to reduce risks. |
| 20 | Satisficing | 6 | Seeks a satisfactory solution rather than the optimal one to save time and resources. |
| 21 | Analytic Hierarchy Process (AHP) | 8 | Breaks down complex decisions into a hierarchy of subproblems, which are analyzed and compared. |
| 22 | Cynefin Framework | 7 | Helps understand the context of a situation to make appropriate decisions in complex environments. |
| 23 | The Five Whys | 7 | Repeatedly asks "Why?" to delve deeper into the root cause of a problem. |
| 24 | Force Field Analysis | 7 | Identifies and analyzes forces for and against a change to make balanced decisions. |
| 25 | Bayesian Decision Theory | 8 | Uses probabilities and statistical methods to make optimal decisions under uncertainty. |
| 26 | Kepner-Tregoe Decision Analysis | 8 | A systematic process for gathering, organizing, and analyzing information to make well-reasoned decisions by evaluating alternatives against objectives. |
| 27 | Decision Trees | 9 | Uses a tree-like model of decisions and their possible consequences, including chance event outcomes and resource costs, to aid in choosing the best alternative. |
| 28 | Scenario Planning | 8 | Explores and analyzes possible future scenarios to enhance strategic decision-making in the face of uncertainty. |
| 29 | Ishikawa (Fishbone) Diagram | 7 | Identifies potential causes of a problem by categorizing factors, aiding in thorough analysis and solution development. |
| 30 | Ladder of Inference | 7 | Illustrates how individuals move from observations to conclusions, helping to improve reasoning and decision-making by reflecting on thought processes. |
| 31 | TRIZ (Theory of Inventive Problem Solving) | 8 | Provides systematic innovation principles to solve complex problems creatively and efficiently. |
| 32 | Stakeholder Analysis | 7 | Identifies and assesses the influence and importance of key people, groups, or entities that may affect or be affected by a decision or project. |
| 33 | Value Chain Analysis | 7 | Analyzes internal firm activities to understand which add value to products or services and could be optimized for competitive advantage. |
| 34 | Porter's Five Forces | 8 | Analyzes industry structure and competitive forces to assess potential profitability and strategic positioning. |
| 35 | Pros and Cons List | 6 | Simple tool for weighing the advantages and disadvantages of options to facilitate straightforward decision-making. |
| 36 | Cost-Effectiveness Analysis | 8 | Compares the relative costs and outcomes of different courses of action to determine the best option in terms of value for money. |
| 37 | Critical Path Method (CPM) | 7 | Identifies the sequence of crucial steps and their timelines in project planning to optimize scheduling and resource allocation. |
| 38 | Real Options Analysis | 8 | Applies financial options theory to investment decisions under uncertainty, allowing flexibility and strategic planning. |
| 39 | Multi-Criteria Decision Analysis (MCDA) | 9 | Evaluates multiple conflicting criteria in decision-making, useful when decisions are complex and involve various stakeholders. |
| 40 | Bounded Rationality | 7 | Recognizes the cognitive limitations of decision-makers and suggests satisficing rather than optimizing, leading to more realistic decisions. |
| 41 | Maximin and Maximax Strategies | 6 | Decision rules to choose the option with the best of the worst (maximin) or the best of the best (maximax) possible outcomes under uncertainty. |
| 42 | Failure Mode and Effects Analysis (FMEA) | 8 | Systematically identifies potential failure modes in a system and their effects to prioritize risk management efforts. |
| 43 | Marginal Decision Rule | 7 | Focuses on the additional benefit vs. the additional cost of a decision to determine if the benefits outweigh the costs incrementally. |
| 44 | Balanced Scorecard | 8 | Translates an organization's strategic objectives into a set of performance measures to guide decision-making and performance management. |
| 45 | SECI Model | 6 | Describes the dynamic process of knowledge creation (Socialization, Externalization, Combination, Internalization) to enhance learning and decision-making in organizations. |
| 46 | DACI Decision-Making Framework | 7 | Clarifies roles in decision-making processes (Driver, Approver, Contributors, Informed) to streamline and improve collaborative decisions. |
| 47 | RAPID Framework | 7 | Defines roles in decision-making (Recommend, Agree, Perform, Input, Decide) to increase accountability and efficiency in organizational decisions. |
| 48 | FAT Model (Facts, Alternatives, Think) | 6 | Encourages decision-makers to focus on gathering facts, considering alternatives, and thoughtful analysis to make informed choices. |
| 49 | Pugh Matrix | 7 | Compares multiple design concepts against a baseline and criteria to identify the best option through a scoring system. |
| 50 | Ethical Decision-Making Models | 8 | Provides structured approaches to address ethical dilemmas by considering principles, values, and consequences. |
| 51 | A3 Problem Solving | 7 | Uses a structured, concise report to solve problems and make decisions in lean management. |
| 52 | Kano Model | 6 | Assesses customer satisfaction with product features to prioritize development decisions. |
| 53 | Theory of Constraints | 8 | Focuses on identifying and addressing the most critical limiting factor (constraint) to improve processes and decision-making. |
| 54 | Agile Decision-Making Framework | 7 | Emphasizes flexibility, collaboration, and responsiveness in decision-making, commonly used in software development. |
| 55 | Heuristic Methods | 6 | Uses simple, efficient rules and mental shortcuts to make quick decisions when complex processing is impractical. |
| 56 | Zero-Based Thinking | 7 | Involves reassessing all decisions and processes from a "starting from scratch" perspective to eliminate outdated practices. |
| 57 | Risk-Adjusted Return Analysis | 7 | Evaluates potential returns on an investment or decision, adjusted for the level of risk involved. |
| 58 | Logic Models | 6 | Visual representations of how a program or decision leads to desired outcomes, helping in planning and evaluation. |
| 59 | Option Pricing Models | 6 | Applies financial models to value options and decisions under uncertainty. |
| 60 | Systems Thinking | 9 | Analyzes complex systems holistically to understand interdependencies and improve strategic decisions. |
| 61 | Kaizen Approach | 8 | Focuses on continuous, incremental improvement in processes and decision-making. |
| 62 | Mind Mapping | 7 | Uses visual diagrams to represent ideas, tasks, or decisions to enhance understanding and creativity. |
| 63 | Net Present Value Analysis | 8 | Calculates the present value of future cash flows to assess the profitability of investments or decisions. |
| 64 | Empathy Mapping | 6 | Helps understand stakeholders' perspectives and feelings to make user-centered decisions. |
| 65 | Blue Ocean Strategy | 8 | Encourages creating uncontested market space through innovative decisions rather than competing in existing markets. |
| 66 | MoSCoW Method | 7 | Prioritizes requirements or tasks into Must have, Should have, Could have, and Won't have categories for decision-making. |
| 67 | RACI Matrix | 7 | Defines roles and responsibilities in processes (Responsible, Accountable, Consulted, Informed) to clarify decision-making authority. |
| 68 | Benefit-Risk Analysis | 8 | Balances the benefits against the risks to make informed decisions, especially in healthcare and pharmaceuticals. |
| 69 | Environmental Scanning | 7 | Monitors external environmental factors to inform strategic decision-making. |
| 70 | SWOT-AHP Hybrid | 8 | Combines SWOT analysis with Analytic Hierarchy Process to prioritize strategic options quantitatively. |
| 71 | PESTLE Analysis | 8 | Assesses macro-environmental factors (Political, Economic, Social, Technological, Legal, Environmental) affecting decisions and strategies. |
| 72 | Influence Diagrams | 6 | Graphical representation of decision problems showing influences among variables for clearer understanding. |
| 73 | Decision Quality Chain | 7 | Ensures each element of a decision (appropriate frame, creative alternatives, meaningful information, clear values, sound reasoning, commitment to action) is of high quality. |
| 74 | Mindful Decision-Making | 7 | Incorporates mindfulness practices to enhance awareness and reduce cognitive biases in decision-making. |
| 75 | Ethical Matrix | 7 | Assesses the ethical dimensions of decisions by evaluating impacts on different stakeholders based on principles like well-being, autonomy, and justice. |
| 76 | Hoshin Kanri | 7 | Aligns an organization's strategic goals with its operational activities through a structured planning process. |
| 77 | Nudge Theory | 7 | Influences behavior and decision-making through subtle suggestions and positive reinforcement without enforcing compliance. |
| 78 | Design Thinking | 9 | Empathizes with users, defines problems, ideates solutions, prototypes, and tests to solve complex problems innovatively. |
| 79 | Lean Six Sigma | 8 | Combines Lean principles and Six Sigma methodology to improve processes by eliminating waste and reducing variation. |
| 80 | SCAMPER Technique | 7 | Encourages creative thinking by Substituting, Combining, Adapting, Modifying, Putting to another use, Eliminating, or Rearranging elements of a problem. |
| 81 | Future Wheel | 7 | Maps out direct and indirect future consequences of a change or decision to anticipate potential outcomes. |
| 82 | SCQA Framework | 6 | Structures communication and problem-solving by outlining the Situation, Complication, Question, and Answer. |
| 83 | KJ Method (Affinity Diagram) | 6 | Organizes ideas and data into groups based on natural relationships to clarify issues and foster consensus. |
| 84 | Jobs to Be Done (JTBD) Framework | 8 | Focuses on understanding customer needs and motivations to make decisions that align with what customers are trying to achieve. |
| 85 | SOAR Analysis | 7 | Strengths, Opportunities, Aspirations, and Results; emphasizes building on strengths and envisioning future outcomes for strategic planning. |
| 86 | CAGE Framework | 6 | Assesses Cultural, Administrative, Geographic, and Economic distances to inform international business decisions. |
| 87 | Laddering Technique | 6 | Explores connections between product attributes, consequences, and personal values to understand underlying motivations in decision-making. |
| 88 | Porter's Diamond Model | 7 | Analyzes the competitive advantage of nations or regions to inform strategic business decisions. |
| 89 | Gap Analysis | 8 | Identifies the gap between the current state and desired future state to prioritize actions for improvement. |
| 90 | Premortem Analysis | 8 | Anticipates potential failures before they occur by imagining a project's failure and working backward to determine what could lead to that failure. |
| 91 | Failure Modes and Effects Criticality Analysis (FMECA) | 7 | Extends FMEA by adding a criticality analysis to prioritize failures based on severity and probability. |
| 92 | Viable System Model (VSM) | 7 | Provides a model for organizational structure to ensure viability and effective decision-making in complex systems. |
| 93 | Reframing Matrix | 7 | Examines business problems from different perspectives (e.g., product, planning, potential, people) to generate comprehensive solutions. |
| 94 | ROAR Framework | 6 | Focuses on Results, Obstacles, Actions, and Resources to aid in decision-making by planning actions while considering challenges and resources. |
| 95 | PEST Analysis | 7 | Assesses Political, Economic, Social, and Technological factors to understand the macro-environment's impact on decisions. |
| 96 | DECIDE Model | 6 | Defines the steps of decision-making: Define, Establish criteria, Consider alternatives, Identify best option, Develop plan, Evaluate decision. |
| 97 | OARP Decision-Making Model | 6 | Outlines steps: Objectives, Alternatives, Risks, and Payoffs to guide structured decisions. |
| 98 | Value-Focused Thinking | 8 | Centers decision-making on fundamental values and objectives rather than alternative options to create better choices. |
| 99 | Critical Question Analysis | 6 | Uses a set of critical questions to evaluate the strength and validity of arguments and decisions. |
| 100 | Appreciative Inquiry | 7 | Focuses on identifying and leveraging an organization's strengths rather than focusing on problems to enhance decision-making. |
| 101 | VUCA Analysis | 7 | Assesses Volatility, Uncertainty, Complexity, and Ambiguity in the environment to inform strategic decision-making. |
| 102 | McKinsey 7S Framework | 7 | Examines seven organizational elements (Strategy, Structure, Systems, Shared Values, Skills, Style, Staff) to ensure alignment and effective decision-making. |
| 103 | Strategy Mapping | 6 | Visualizes organizational goals and the linkages between them to improve strategic planning and decision-making. |
| 104 | ADKAR Model | 6 | Guides change management through Awareness, Desire, Knowledge, Ability, and Reinforcement to support decision implementation. |
| 105 | OKR Framework | 7 | Aligns decisions with organizational Objectives and Key Results to ensure focus and measurable outcomes. |
| 106 | BCG Matrix | 7 | Categorizes business units or products into Cash Cows, Stars, Question Marks, and Dogs to inform investment decisions. |
| 107 | Ansoff Matrix | 7 | Helps decide on growth strategies by analyzing product and market combinations for strategic decision-making. |
| 108 | Design Sprint Framework | 7 | Accelerates problem-solving and decision-making through a five-phase process of idea generation, prototyping, and testing. |
| 109 | PACED Decision-Making Model | 6 | Offers a structured approach: Problem, Alternatives, Criteria, Evaluate, Decide to simplify complex decisions. |
| 110 | SMART Goals Framework | 7 | Ensures decisions and objectives are Specific, Measurable, Achievable, Relevant, and Time-bound for effective planning. |
| 111 | OGSM Framework | 6 | Connects Objectives, Goals, Strategies, and Measures to align decisions with organizational vision and execution. |
| 112 | Socratic Questioning Technique | 6 | Enhances critical thinking by systematically questioning assumptions and exploring underlying beliefs in decision-making. |
| 113 | Real-Time Decision-Making Frameworks | 7 | Facilitates immediate decisions based on live data to enhance responsiveness in dynamic environments. |
| 114 | Core Competency Analysis | 6 | Identifies unique strengths to inform decisions that leverage an organization's competitive advantages. |
| 115 | Soft Systems Methodology (SSM) | 6 | Addresses complex, unstructured problems through systemic inquiry to inform better decision-making. |
| 116 | Four Frame Model | 6 | Analyzes organizations from Structural, Human Resource, Political, and Symbolic frames to improve decision-making. |
| 117 | TOWS Matrix | 7 | Extends SWOT by matching internal strengths and weaknesses with external opportunities and threats to generate strategic options. |
| 118 | Crisis Decision-Making Models | 7 | Provides frameworks for making effective decisions under high-pressure and time-sensitive situations. |
| 119 | Optimal Stopping Theory | 6 | Determines the ideal time to make a decision to maximize payoff in uncertain conditions. |
| 120 | Multi-Attribute Utility Theory (MAUT) | 8 | Assists in making choices involving trade-offs among multiple attributes by quantifying preferences. |
| 121 | Expected Utility Theory | 7 | Guides decision-making under uncertainty by choosing options with the highest expected utility. |
| 122 | Action Priority Matrix | 6 | Prioritizes actions based on the impact and effort to focus on decisions that maximize results with minimal effort. |
| 123 | Portfolio Analysis | 7 | Evaluates the performance and risk of a portfolio of investments or projects to inform allocation decisions. |
| 124 | Cognitive Mapping | 6 | Visualizes individual or group thinking on a problem to clarify complex decisions through shared understanding. |
| 125 | Anticipatory Thinking | 7 | Foresees potential future events and conditions to make proactive decisions. |
| 126 | System Dynamics Modeling | 7 | Uses feedback loops and time delays to understand the behavior of complex systems over time, aiding in strategic decision-making. |
| 127 | Theory of Planned Behavior | 7 | Considers attitudes, subjective norms, and perceived behavioral control to understand and predict individual decision-making. |
| 128 | Fast and Frugal Heuristics | 7 | Simplifies decision-making using simple rules that require minimal information, often leading to effective outcomes quickly. |
| 129 | Nominal Group Technique (NGT) | 7 | Facilitates group decision-making by gathering ideas and prioritizing them through structured ranking and voting. |
| 130 | Consensus Decision-Making | 6 | Achieves group agreement in decision-making processes, ensuring all members support the final decision. |
| 131 | DMADV Framework | 7 | Define, Measure, Analyze, Design, Verify; a Six Sigma methodology for designing processes or products to meet customer needs and process performance. |
| 132 | Behavioral Economics Frameworks | 8 | Incorporates insights from psychology into economics to better understand decision-making under uncertainty and irrationality. |
| 133 | Multi-Objective Decision Analysis (MODA) | 8 | Evaluates and balances multiple conflicting objectives in decision-making, providing a structured approach to trade-offs. |
| 134 | GROW Model | 7 | Goal, Reality, Options, Will; a coaching framework guiding individuals through decision-making by defining goals and exploring options. |
| 135 | Fuzzy Logic Decision-Making | 7 | Utilizes degrees of truth rather than binary true/false to handle uncertainty in complex decision scenarios. |
| 136 | Expected Regret Theory | 7 | Considers potential regret in decision-making to minimize future dissatisfaction by evaluating opportunity losses. |
| 137 | Technology Acceptance Model (TAM) | 6 | Predicts how users accept and use technology, informing decisions in technology implementation and adoption. |
| 138 | SFA Framework | 7 | Suitability, Feasibility, Acceptability; assesses strategic options to ensure they align with goals, are practical, and are agreeable to stakeholders. |
| 139 | Decision Framing | 7 | Emphasizes the influence of presentation and context on decision outcomes to improve decision-making clarity and effectiveness. |
| 140 | Ethical Triangle | 7 | Integrates principles, consequences, and virtues to guide ethical decision-making in complex situations. |
| 141 | Shifting Perspectives Model | 6 | Encourages viewing problems from multiple viewpoints to challenge assumptions and enhance decision-making. |
| 142 | Stakeholder Salience Model | 7 | Prioritizes stakeholders based on power, legitimacy, and urgency to inform engagement strategies in decision-making. |
| 143 | The Disney Creative Strategy | 6 | Approaches problems from Dreamer, Realist, and Critic perspectives to foster creativity and thorough decision-making. |
| 144 | Scenario Comparison Framework | 6 | Evaluates multiple potential future scenarios side-by-side to inform strategic decisions and planning. |
| 145 | Analytical Network Process (ANP) | 8 | Generalizes AHP by incorporating interdependencies among decision elements for comprehensive decision analysis. |
| 146 | Portfolio Decision Analysis (PDA) | 7 | Facilitates decisions involving portfolios of projects or investments, evaluating them collectively for optimal allocation. |
| 147 | Socio-Technical Systems Approach | 7 | Considers both social and technical factors in organizational decision-making to optimize system performance. |
| 148 | Technology Roadmapping | 6 | Aligns technological capabilities with business objectives over time to guide strategic decision-making. |
| 149 | Value Engineering | 6 | Enhances value by analyzing functions and reducing costs without sacrificing quality, aiding in cost-effective decisions. |
| 150 | Leverage Points Analysis | 7 | Identifies strategic points within a system where interventions can lead to significant improvements in outcomes. |
| 151 | Likelihood-Consequence Matrix | 6 | Assesses risks by evaluating the likelihood and impact of potential events to inform risk management decisions. |
| 152 | Group Decision Support Systems (GDSS) | 7 | Employs technology to facilitate group decision-making processes, enhancing collaboration and efficiency. |
| 153 | Dialectical Inquiry | 6 | Involves structured debate of opposing viewpoints to uncover assumptions and improve the quality of decisions. |
| 154 | Devil's Advocacy Technique | 7 | Assigns a team member to challenge ideas to identify weaknesses and strengthen decision outcomes. |
| 155 | Consensus Mapping | 6 | Visualizes areas of agreement among group members to facilitate consensus in decision-making. |
| 156 | Theory of Reasoned Action | 6 | Explores how individual behavior is driven by intentions shaped by attitudes and social norms, aiding in predicting decisions. |
| 157 | Expected Value Decision Rule | 7 | Calculates expected values by weighting outcomes with probabilities to guide decisions under uncertainty. |
| 158 | Three Horizons Framework | 6 | Encourages balancing current performance with future opportunities by focusing on innovation over three time horizons. |
| 159 | Emotional Intelligence Frameworks | 7 | Incorporates emotional awareness and regulation to enhance interpersonal dynamics and decision-making effectiveness. |
| 160 | Johari Window | 6 | Helps improve self-awareness and mutual understanding within groups, fostering better collaborative decisions. |
| 161 | S.M.A.R.T Goals Framework | 7 | Ensures objectives are Specific, Measurable, Achievable, Relevant, and Time-bound to guide effective decision-making. |
| 162 | Value Stream Mapping | 7 | Visualizes steps needed to deliver a product or service to identify and eliminate waste, streamlining decision-making processes. |
| 163 | Red Teaming | 7 | Challenges plans and strategies from an adversarial perspective to identify vulnerabilities and improve decision resilience. |
| 164 | Theory U | 6 | Guides leaders through a process of letting go of old patterns and embracing emerging future possibilities for innovative decision-making. |
| 165 | Soft Systems Methodology (SSM) | 6 | Addresses complex, ill-structured problems using systemic inquiry for improved decision-making in organizational contexts. |


### Step 3 - Gathering Intelligence with External Knowledge Stores